# 计数器

计数器是应用中最常见的功能之一，如：
+ 阅读应用会用计数器记录每本书、每篇文章被阅读的次数。
+ 应用商店会用计数器记录每个应用被下载的次数和付费购买应用的人数。
+ 视频应用、音乐应用会用计数器记录视频和音乐被播放的次数。
+ 为了保护用户的财产安全，银行应用可能会在在后台用计数器记录每个账户的登陆失败次数，并在需要的时候锁定账户以防止密码被暴力破解。  

下面将会使用 Redis 实现计数器，从而对系统或用户的某些操作进行技术，有两种方案：
+ 使用字符串键
+ 使用哈希键

### 使用字符串键

整体逻辑比较简单，和上一节“自增数字 ID”比较像，不做过多解释。

In [3]:
class Counter:

    def __init__(self, client, key):
        self.client = client
        self.key = key

    def increase(self, n=1):
        """
        将计数器的值加上指定的数字。
        """
        return self.client.incr(self.key, n)

    def decrease(self, n=1):
        """
        将计数器的值减去指定的数字。
        """
        # DECRBY key 1
        return self.client.decr(self.key, n)

    def get(self):
        """
        返回计数器的当前值。
        """
        value = self.client.get(self.key)
        return 0 if value is None else int(value)

    def reset(self, n=0):
        """
        将计数器的值重置为参数n指定的数字，并返回计数器在重置之前的旧值。
        参数n是可选的，若省略则默认将计数器重置为0。
        """
        value = self.client.set(self.key, n, get=True)
        return 0 if value is None else int(value)

### 使用哈希键

也和上一节“自增数字 ID”比较像，不做过多解释，就是把多个相关的计数器放到同一个哈希键中进行管理，如访问计数器、下载计数器、付费计数器。

另外作者还提到了，对于一些使用哈希键存储的文章信息的键，比如，一个键中包含了文章的标题、正文、作者、发布日期等，这时比起使用别的字符串存储文章的浏览量，更好的做法是把浏览量也包含在相同的哈希键中，在文章被阅读时，更新它的浏览量。

In [5]:
class HashCounter:

    def __init__(self, client, key, name):
        """
        创建一个哈希键计数器对象。
        其中key参数用于指定包含多个计数器的哈希键的键名，
        而name参数则用于指定具体的计数器在该键中的名字。
        """
        self.client = client
        self.key = key
        self.name = name

    def increase(self, n=1):
        """
        将计数器的值加上指定的数字。
        """
        return self.client.hincrby(self.key, self.name, n)

    def decrease(self, n=1):
        """
        将计数器的值减去指定的数字。
        """
        # Redis并没有HDECRBY，所以只能通过传给HINCRBY负数进行减少
        return self.client.hincrby(self.key, self.name, 0-n)

    def get(self):
        """
        返回计数器的当前值。
        """
        value = self.client.hget(self.key, self.name)
        if value is None:
            return 0
        else:
            return int(value)

    def reset(self, n=0):
        """
        将计数器的值重置为参数n指定的数字，并返回计数器在重置之前的旧值。
        参数n是可选的，若省略则默认将计数器重置为0。
        """
        tx = self.client.pipeline()
        tx.hget(self.key, self.name)  # 获取旧值
        tx.hset(self.key, self.name, n)  # 设置新值
        old_value, _ = tx.execute()
        if old_value is None:
            return 0
        else:
            return int(old_value)